# Blocos `with` e contextos

## 1. Revisão de `with` e arquivos

Como vimos, é recomendado fazer acessos a arquivos por uma construção usando blocos `with`:

In [ ]:
with open('teste.txt', encoding='utf-8') as file:
    for i, line in enumerate(file.readlines()):
        print(f'{i:02}', line, end='')

Como discutido, isto garante que o arquivo seja fechado mesmo que o código dentro do `with` seja interrompido antes de seu final, seja por um término prematuro ou por uma exceção.

Neste caso, o funcionamento é o seguinte:

1. A função `open` é executada e o resultado (valor retornado por `open`) é colocado na variável `file`.
2. O código dentro do `with` é executado.
3. Quando o código dentro do `with` termina (seja normalmente ou por qualquer interrupção), a operação `close` é executada sobre o `file`, antes de passar a execução para o código que segue o `with`.

## 2. Gerenciadores de contexto

Mas este não é o único propósito do `with`. Na verdade, o `with` é um comando a ser usado em conjunto com **gerenciadores de contexto**.

Um gerenciador de contexto é um objeto que implementa dois métodos mágicos:

- O método `__enter__`, que serve para indicar que estamos entrando em um contexto específico, e
- O método `__exit__`, que indica que estamos saindo do contexto.

Um código como

```
with obj as var:
    commands
```

é executado da seguinte forma:

1. O método `__enter__` é executado sobre `obj`.
2. A referência retornada por esse método é colocada na variável `var`. A parte `as var` pode ser omitida, caso em que a referência retornada por `__enter__` será descartada.
3. Os comandos em `commands` são executados
4. Quando a execução de `commands` termina ou é interrompida, o método `__exit__` é executado sobre `obj`.
5. Por fim, a execução continua no código após o `with` ou no tratamento de uma exceção que tenha ocorrido.

Para entender melhor esse processo, vamos estudar os detalhes dos dois métodos mágicos.

## 3. O método `__enter__`

Este método não recebe nenhum parâmetro (a não ser o `self` referente ao objeto sobre o qual foi chamado). A função deste método é fazer a inicialização do contexto e portanto o que ele faz depende do que queremos que o contexto represente.

Um fator importante é que o método precisa retornar um objeto, que terá sua referência colocada na variável usada na parte `as`. Se o seu tipo de contexto não precisar dessa variável, você pode retornar `None`.

Por exemplo, no caso do `with` com `open`, a função `open` abre o arquivo e retorna um objeto do tipo `TextIOWrapper`. Esse objeto implementa o protocolo de gerenciador de contexto, e o seu método `__enter__` simplesmente retorna o próprio objeto, que pode então ser usado nas operações de arquivo dentro do bloco `with`.

## 4. O método `__exit__`

Este método é um pouco mais complicado. Sua função é realizar as operações que garantem que o contexto foi terminado.

No caso de `with` com `open`, ele precisa apenas realizar a operação `close` no arquivo.

A interface para esse método é a seguinte:

```
__exit__(self, exception_type, exception_value, traceback)
```

Além do objeto gerenciador de contexto, o método recebe informações sobre uma possível exceção que tenha interrompido a execução do bloco `with`: seu tipo (classe) em `exception_type`, o objeto de exceção em `exception_value` e informação sobre o ponto onde a exceção ocorreu em `traceback`.  Se nenhuma exceção ocorreu esse três parâmetros são `None`.

Caso o bloco tenha sido interrompido por uma exceção, o `__exit__` deve retornar um valor falso (por exemplo, `False`, ou `None`) para que a exceção seja tratada posteriormente pelo código. Se o próprio `__exit__` trata da exceção, ele deve retornar `True`, para que o sistema não tente tratar novamente da exceção.

Caso não ocorra exceção, o valor de retorno do `__exit__` é ignorado, e portanto irrelevante.

As informações sobre exceção são fornecidas para o caso das operações de término de contexto dependerem do tipo de exceção. Em muitos casos não precisamos usar essas informações.

## 5. Exemplo, versão 1

Vamos ver agora um exemplo simples, para consolidar as informações.

In [ ]:
class NotEnoughMoneyException(ValueError):
    pass


class LockedException(Exception):
    pass


class Safe:
    
    def __init__(self, initial = 0):
        assert initial >= 0, 'Cannot start with negative amount'
        self._amount = initial
        self._locked = True
       
    def is_locked(self):
        return self._locked
    
    def _verify_access(self):
        if self._locked: 
            raise LockedException()
        
    def get_amount(self):
        self._verify_access()
        return self._amount
    
    
    def add(self, value):
        self._verify_access()
        assert value >= 0, 'Cannot add negative amount.'
        self._amount += value
        
        
    def remove(self, value):
        self._verify_access()
        assert value >= 0, 'Cannot remove negative amount.'
        if value > self._amount:
            raise NotEnoughMoneyException()
        self._amount -= value
        
        
    def __enter__(self):
        self._locked = False
        return self
    
    
    def __exit__(self, exc_type, exc_val, tb):
        self._locked = True
        if exc_type is not None:
            print('Something went wrong.')
            return False

In [ ]:
sf1 = Safe(1000)

In [ ]:
sf1.get_amount()

In [ ]:
with sf1 as sf:
    print('Current amount is ', sf.get_amount())

In [ ]:
sf1.get_amount()

In [ ]:
with sf1:
    sf1.remove(500)
    print('Remaining amount is', sf1.get_amount())

In [ ]:
with sf1:
    print(f'The safe is {"not " if not sf1.is_locked() else ""}locked')
    sf1.remove(700)
    print('Remaining amount is', sf1.get_amount())

In [ ]:
print(f'The safe is {"not " if not sf1.is_locked() else ""}locked')

In [ ]:
from random import randint
with Safe(100) as sf:
    print('Initial amount is', sf.get_amount())
    for i in range(10):
        how_many = randint(1, 30)
        print(f'Withdrawing {how_many}')
        sf.remove(how_many)
        print('New amount is', sf.get_amount())
print('This is the end')

## 6. Exemplo, versão 2

O exemplo anterior mostra como uma classe pode ser expandida para ser também um gerenciador de contexto. No entanto, a solução acima apresenta o problema de que a classe desenvolvida está lidando com duas responsabilidade: Cuidar da quantidade de dinheiro guardada e cuidar do acesso a esse dinheiro.

Juntar duas responsabilidades na mesma classe não é uma boa forma de solucionar um problema. É melhor separarmos as responsabilidades em duas classes distintas. Esse é o princípio de programação denominado **separação de responsabilidades**.

Vamos aplicar esse princípio para refazer o código acima.

In [ ]:
class NotEnoughMoneyException(ValueError):
    pass


class Wallet:
    
    def __init__(self, initial = 0):
        assert initial >= 0, 'Cannot start with negative amount'
        self._amount = initial

    def get_amount(self):
        return self._amount
    
    
    def add(self, value):
        assert value >= 0, 'Cannot add negative amount.'
        self._amount += value
        
        
    def remove(self, value):
        assert value >= 0, 'Cannot remove negative amount.'
        if value > self._amount:
            raise NotEnoughMoneyException()
        self._amount -= value



class Safe:
    
    def __init__(self, wallet):
        self._wallet = wallet
        self._locked = True

        
    def is_locked(self):
        return self._locked

            
    def __enter__(self):
        self._locked = False
        return self._wallet
    
    
    def __exit__(self, exc_type, exc_val, tb):
        self._locked = True
        if exc_type is not None:
            print('Something went wrong.')
            return False

Veja como ao separarmos as responsabilidades em classes distintas os códigos ficam mais claros. O uso muda um pouco: ao entramos no contexto, recebemos uma `Wallet`, sobre a qual fazemos as opearações. Como os métodos de acesso ao dinheiro só estão disponíveis na `Wallet`, e como só podemos acessar uma `Wallet` que está num `Safe` através do contexto, não precisamos nos preocupar com um acesso desautorizado ao dinheiro.

In [ ]:
sf2 = Safe(Wallet(1000))

In [ ]:
sf2.get_amount()

In [ ]:
sf2.is_locked()

In [ ]:
with sf2 as wallet:
    print('Current amount is', wallet.get_amount())
    wallet.remove(500)
    print('Current amount is', wallet.get_amount())    

In [ ]:
with sf2 as wallet:
    print('Safe is now locked?', sf2.is_locked())
    wallet.remove(700)

In [ ]:
sf2.is_locked()

## 7. Mais opções

O módulo `contextlib` apresenta algumas facilidades para desenvolver gerenciadores de contexto para alguns casos simples. Veja a [documentação](https://docs.python.org/3/library/contextlib.html).

# Exercício

Escreva uma classe denominada `MessageBracket` para servir de gerenciador de contexto de forma que, quando um objeto dessa classe é usado num `with`, ele executa o código do `with` normalmente, mas colocando uma mensagem (passada como primeiro parâmetro na construção do objeto) antes do bloco do `with` e outra mensagem (passada como segundo parâmetro na construção do objeto) após a execução do bloco do `with`. Ele também pode receber uma parâmetro opcional, de nome `err_msg` com a mensagem a escrever caso o bloco tenha sido interrompido por alguma exceção. Veja exemplo de uso abaixo.

In [ ]:
with MessageBracket('Hi!', 'Bye!', err_msg='Problems...'):
    for i in range(3):
        x = int(input('Give me an int'))
        print(f'You gave {x}')